<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/IPA_practice01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = "https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"

Note: Options to use

+ Lower case only
+ Voicing: voiced or voiceless
+ Place: bilabial, labio-dental, dental, alveolar, palato-alveolar, velar, gottal
+ Centrality: central or lateral
+ Manner: stop, fricative, affricate, approximant

In [ ]:
#@markdown IPA practice with points

import pandas as pd
import random
from IPython.display import display, clear_output
import ipywidgets as widgets

url = "https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"

# Load the CSV data
def load_data():
    return pd.read_csv(url, encoding='utf-8')

# Display a random IPA symbol and quiz the user
def quiz(df, score):
    clear_output(wait=True)
    row = df.sample(1).iloc[0]
    ipa_symbol = row['IPA']
    properties = ['Voicing', 'Place', 'Centrality', 'Oro-nasal', 'Manner']
    question_property = random.choice(properties)

    print(f"IPA Symbol: {ipa_symbol}")
    print(f"What is the {question_property.lower()} of this IPA symbol?")

    input_text = widgets.Text(
        value='',
        placeholder='Type your answer here',
        description='Answer:',
        disabled=False
    )

    def check_answer(b):
        if input_text.value.lower() == str(row[question_property]).lower():
            score[0] += 1
            print(f"Correct! Total points: {score[0]}")
        else:
            print(f"Wrong! The correct answer was {row[question_property]}. Total points: {score[0]}")
        ask_continue(score)

    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(check_answer)

    display(input_text, submit_button)

# Ask whether to continue or quit the quiz
def ask_continue(score):
    continue_button = widgets.Button(description="Continue")
    quit_button = widgets.Button(description="Quit")

    def continue_quiz(b):
        quiz(df, score)

    def quit_quiz(b):
        clear_output(wait=True)
        print(f"Well-practiced! Your total points are: {score[0]}")

    continue_button.on_click(continue_quiz)
    quit_button.on_click(quit_quiz)

    display(continue_button, quit_button)

# Start the quiz
df = load_data()
score = [0]  # Using list to keep reference in nested functions
quiz(df, score)


# Graio

In [ ]:
!pip install gradio

In [ ]:
"https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"

In [ ]:
import pandas as pd
import random
import gradio as gr

# Load the CSV data from the provided URL
def load_data():
    url = "https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"
    return pd.read_csv(url, encoding='utf-8')

df = load_data()  # Load data once at the start

def generate_question():
    row = df.sample(1).iloc[0]
    properties = ['Voicing', 'Place', 'Centrality', 'Oro-nasal', 'Manner']
    selected_property = random.choice(properties)
    question = f"IPA Symbol: {row['IPA']}\nWhat is the {selected_property.lower()} of this IPA symbol?"
    return question, row[selected_property].lower(), selected_property.lower()

def quiz_function(user_answer, score, question_info):
    correct_answer = question_info['answer']
    property_name = question_info['property']
    if user_answer.lower() == correct_answer:
        score[0] += 1
        result = f"Correct! The {property_name} is '{correct_answer}'. Total points: {score[0]}"
    else:
        result = f"Wrong! The correct {property_name} was '{correct_answer}'. Total points: {score[0]}"
    # Generate a new question for the next round
    new_question_info = {'question': generate_question()}
    return result, new_question_info

def gradio_interface():
    score = [0]  # Use a list to maintain score across function calls
    question_info = {'question': '', 'answer': '', 'property': ''}

    def initialize_quiz():
        question_info['question'], question_info['answer'], question_info['property'] = generate_question()
        return question_info['question'], "", ""

    def submit_answer(user_answer):
        result, new_question_info = quiz_function(user_answer, score, question_info)
        question_info.update(new_question_info)
        return result, question_info['question'], ""

    def continue_quiz():
        question_info['question'], question_info['answer'], question_info['property'] = generate_question()
        return question_info['question'], "", ""

    def quit_quiz():
        return f"Quiz ended. Your total score: {score[0]} points.", None, None

    with gr.Blocks() as app:
        question_label = gr.Textbox(label="Question")
        answer_input = gr.Textbox(label="Your Answer", placeholder="Type your answer here")
        submit_button = gr.Button("Submit")
        output_label = gr.Textbox(label="Result")

        continue_button = gr.Button("Continue")
        quit_button = gr.Button("Quit")

        submit_button.click(fn=submit_answer, inputs=[answer_input], outputs=[output_label, question_label, answer_input])
        continue_button.click(fn=continue_quiz, outputs=[question_label, answer_input])
        quit_button.click(fn=quit_quiz, outputs=[output_label])

        # Initialize the first question
        question_label.value, _, _ = initialize_quiz()

    return app

app = gradio_interface()
app.launch()


In [ ]:
import pandas as pd
import random
import gradio as gr

# Load the CSV data from the provided URL
def load_data():
    url = "https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"
    return pd.read_csv(url, encoding='utf-8')

df = load_data()  # Load data once at the start

# Function to generate a question
def generate_question():
    row = df.sample(1).iloc[0]
    properties = ['Voicing', 'Place', 'Centrality', 'Oro-nasal', 'Manner']
    selected_property = random.choice(properties)
    question = f"IPA Symbol: {row['IPA']}\nWhat is the {selected_property.lower()} of this IPA symbol?"
    return question, row[selected_property].lower(), selected_property.lower()

# Function to handle answer submission and update score
def quiz_function(user_answer, score, question_info):
    correct_answer = question_info['answer']
    property_name = question_info['property']
    if user_answer.lower() == correct_answer:
        score[0] += 1
        result = f"Correct! The {property_name} is '{correct_answer}'. Total points: {score[0]}"
    else:
        result = f"Wrong! The correct {property_name} was '{correct_answer}'. Total points: {score[0]}"
    # Generate a new question for the next round
    new_question_info = {'question': generate_question()}
    return result, new_question_info

# Gradio interface function
def gradio_interface():
    score = [0]  # Track score across rounds
    question_info = {'question': '', 'answer': '', 'property': ''}

    # Initialize the quiz with the first question
    def initialize_quiz():
        question_info['question'], question_info['answer'], question_info['property'] = generate_question()
        return question_info['question'], "", ""

    # Handle answer submission
    def submit_answer(user_answer):
        result, new_question_info = quiz_function(user_answer, score, question_info)
        question_info.update(new_question_info)
        return result, question_info['question'], ""

    # Continue with a new question without checking the previous answer
    def continue_quiz():
        question_info['question'], question_info['answer'], question_info['property'] = generate_question()
        return question_info['question'], "", ""

    # End the quiz and display the total score
    def quit_quiz():
        return f"Quiz ended. Your total score: {score[0]} points.", None, None

    # Set up the Gradio interface
    with gr.Blocks() as app:
        question_label = gr.Textbox(label="Question", interactive=False)  # Read-only for the question
        answer_input = gr.Textbox(label="Your Answer", placeholder="Type your answer here")
        output_label = gr.Textbox(label="Result", interactive=False)  # Read-only for the result
        submit_button = gr.Button("Submit")
        continue_button = gr.Button("Continue")
        quit_button = gr.Button("Quit")

        # Define button actions
        submit_button.click(fn=submit_answer, inputs=[answer_input], outputs=[output_label, question_label, answer_input])
        continue_button.click(fn=continue_quiz, outputs=[question_label, answer_input])
        quit_button.click(fn=quit_quiz, outputs=[output_label])

        # Define layout
        app.add(question_label)
        app.add(answer_input)
        app.add(submit_button)  # Submit button placed correctly after the answer input
        app.add(output_label)
        with gr.Row():  # Continue and Quit buttons in the same row
            app.add(continue_button)
            app.add(quit_button)

        # Initialize the first question
        question_label.value, _, _ = initialize_quiz()

    return app

# Run the Gradio interface
app = gradio_interface()
app.launch()
